In [51]:
import os
import json
import pandas as pd
import traceback

In [52]:
#from langchain.chat_models import ChatOpenAI

# Using the google AI studio api
from langchain_google_genai import ChatGoogleGenerativeAI


In [29]:
from dotenv import load_dotenv

load_dotenv()

True

In [30]:
KEY = os.getenv("GOOGLE_API_KEY")

In [70]:
#llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5) 
# temperature range is 0 to 2

llm = ChatGoogleGenerativeAI(model = "gemini-pro", temperature=0.5)

In [71]:
llm

ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.5, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002683293FD10>, default_metadata=())

In [55]:
#from langchain.llms import OpenAI
#from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [98]:
RESPONSE_JSON = {
    "1" : {
        "mcq" : "multiple choice questions",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",
    },
    "2" : {
        "mcq" : "multiple choice questions",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
    },
    "3" : {
        "mcq" : "multiple choice questions",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
    },
    "4" : {
        "mcq" : "multiple choice questions",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
    },
    "5" : {
        "mcq" : "multiple choice questions",
        "options" : {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
    }
}

In [140]:
TEMPLATE = """
Text : {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON and use it as a guide. \
Ensure to make {number} MCQs
{response_json}

"""

In [141]:
# designing the prompt using prompt_template
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [142]:
# creating llm chain
# used for connecting various component here 1st is llm and other is prompt
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)

In [143]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple choice quiz for {subject} students. \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
if the quiz is not as per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students level. \
Quiz_MCQs:
{quiz}

check from an expert English writer of the above quiz:
"""

In [144]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template= TEMPLATE2
)

In [145]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)


In [146]:
generate_evaluation_chain = SequentialChain(chains = [quiz_chain,review_chain], input_variables = ["text","number","subject","tone","response_json"],
                                            output_variables = ["quiz", "review"], verbose = True)

In [147]:
file = open(r"C:\Users\anshr\mcqgen\data.txt", 'r')
text = file.read()

In [148]:
# Serialize the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}}, "4": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}}, "5": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}}}'

In [149]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [ ]:
# How to setup Token usage Tracking in Langchain
with get_openai_callback() as cb:
    response = generate_evaluation_chain(
        {
            "text" : text,
            "number" : NUMBER,
            "subject" : SUBJECT,
            "tone" : TONE,
            "response_json" : json.dumps(RESPONSE_JSON)
        }
    )

In [78]:
# Implementing the custom callback Handler for Token Usage Tracking
from langchain.callbacks.base import BaseCallbackHandler

class TokenUsageCallbackHandler(BaseCallbackHandler):
    def __init__(self):
        self.total_tokens = 0

    def on_llm_end(self,response, **kwargs):
        self.total_tokens += response.usage_metadata.get('total_tokens', 0)

    def get_total_tokens(self):
        return self.total_tokens

In [150]:
# initialize the custom callback handler
token_usage_callback = TokenUsageCallbackHandler()

# input data
input_data = {
    "text": text,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
}

# Execute the chain with the callback
response = generate_evaluation_chain(input_data)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, proba

In [80]:
total_tokens = token_usage_callback.get_total_tokens()
print(f"Total tokens used: {total_tokens}")

Total tokens used: 0


In [151]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing machine learning.

In [156]:
quiz_str = response.get("quiz")

In [161]:
print(quiz_str)

{
"1": {"mcq": "What is the primary focus of machine learning?", "options": {"a": "Developing algorithms that learn from data", "b": "Creating artificial intelligence systems", "c": "Improving computer vision", "d": "None of the above"}, "correct": "a"},
"2": {"mcq": "Which field has significantly improved the performance of neural networks?", "options": {"a": "Natural language processing", "b": "Deep learning", "c": "Predictive analytics", "d": "Data mining"}, "correct": "b"},
"3": {"mcq": "What is the application of machine learning to business problems known as?", "options": {"a": "Data science", "b": "Predictive analytics", "c": "Artificial intelligence", "d": "Machine learning engineering"}, "correct": "b"},
"4": {"mcq": "Which of the following is NOT a foundation of machine learning?", "options": {"a": "Statistics", "b": "Computer science", "c": "Mathematical optimization", "d": "Data mining"}, "correct": "b"},
"5": {"mcq": "What is the theoretical framework used to describe mach

In [160]:
quiz_str = quiz_str.replace('RESPONSE_JSON = ', '')

In [159]:
type(quiz_str)

str

In [162]:
quiz = json.loads(quiz_str)

In [ ]:
print(f"Total tokens:{cb.total_tokens}")
print(f"Total tokens:{cb.prompt_tokens}")
print(f"Total tokens:{cb.completion_tokens}")
print(f"Total tokens:{cb.total_cost}")

In [164]:
quiz_table_data = []
for key,value in quiz.items():
    mcq = value['mcq']
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ" : mcq, "Choices": options, "Correct": correct})

In [166]:
df = pd.DataFrame(quiz_table_data)

In [179]:
df['Choices'][0]

'a : Developing algorithms that learn from data | b : Creating artificial intelligence systems | c : Improving computer vision | d : None of the above'